In [7]:
import pandas as pd
import requests
import json
import os

# Authenticate with Bayern Cloud

In [3]:
# Load Bayern Cloud API key from environment variables
BAYERN_CLOUD_API_KEY = os.getenv('BAYERN_CLOUD_API_KEY')

# Load occupancy data

In [10]:
def load_and_save_occupancy_data_for_location(
    location_slug: str,
    save_file_path: str = 'outputs'
):
    API_endpoint = f'https://data.bayerncloud.digital/api/v4/endpoints/list_occupancy/{location_slug}'

    request_params = {
        'token': BAYERN_CLOUD_API_KEY
    }

    response = requests.get(API_endpoint, params=request_params)
    response_json = response.json()

    os.makedirs(save_file_path, exist_ok=True)

    # Save response to JSON file
    with open(f'{save_file_path}/occupancy_data_{location_slug}.json', 'w') as outfile:
        json.dump(response_json, outfile)

    return


In [11]:
location_slug = "parkplatz-graupsaege-1"
load_and_save_occupancy_data_for_location(location_slug)